In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px
# from plotly import graph_objects as go

import matplotlib
# import plotly
import sklearn

print("package版本信息：")
print("numpy:      ", np.__version__)
print("pandas:     ", pd.__version__)
print("matplotlib: ", matplotlib.__version__)
print("sklearn:    ", sklearn.__version__)
print("seaborn:    ", sns.__version__)
# print("plotly:     ", plotly.__version__)

package版本信息：
numpy:       1.22.3
pandas:      1.4.2
matplotlib:  3.5.1
sklearn:     1.0.2
seaborn:     0.11.2


In [2]:
# 设置显示所有的列
pd.options.display.max_columns = None
# 设置显示所有的行
pd.options.display.max_rows = None

# 阻止waring显示
import warnings
warnings.filterwarnings('ignore')

# jupyter notebook设置同一个cell打印多个结果
from IPython.display import display
# 然后使用
# display("a")
# display("b")

In [4]:
%cd ..

/Users/danielzhang/Documents/Python-Projects/DataAnalysis


In [5]:
%pwd

'/Users/danielzhang/Documents/Python-Projects/DataAnalysis'

主要讨论使用sklearn构建机器学习的流水线，实现一步到位的处理过程。

参考文档：
+ [sklearn -> 6.1. Pipelines and composite estimators](https://scikit-learn.org/stable/modules/compose.html#pipelines-and-composite-estimators)

# 流水线

sklearn中流水线功能主要由 `sklearn.pipeline` 这个模块提供，主要是如下的`Pipeline`类

`Pipeline(steps, *, memory=None, verbose=False)`类
+ 主要参数
  + `steps`：list of (name, transformer) tuples，其中每个transformer都必须实现`fit`/`transform`方法，最后一个必须是estimator

此外，还有一个`make_pipeline()`函数，用于快速创建一个`Pipeline`对象。

In [8]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [7]:
estimators = [('reduce_dim', PCA()), ('clf', SVC())]
pipe = Pipeline(estimators)
pipe

Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC())])

In [10]:
# 或者使用 make_pipeline，会自动生成每个步骤的名字
pipe2 = make_pipeline(PCA(), SVC())
pipe2

Pipeline(steps=[('pca', PCA()), ('svc', SVC())])

# 多类型数据转换

通常数据的特征会有多种类型，比如同时含有类别型和数值型特征，此时在pipeline中添加数据预处理步骤时，需要对不同的特征使用不同的变换器，比如对于类别型使用one-hot编码，对于计数型特征使用分箱KBinarize。  

此时就需要`sklearn.compose`模块中的`ColumnTransformer`类的协助：   

`ColumnTransformer(transformers, *, remainder='drop', sparse_threshold=0.3, n_jobs=None, transformer_weights=None, verbose=False, verbose_feature_names_out=True)`
+ `transformers`：   
  List of (name, transformer, columns) tuples，用于指定各个列的转换器
  + name：str，转换器的名称
  + transformer：estimator对象 或者 {‘drop’, ‘passthrough’}，estimator时使用对应的转换器，'drop'表示该列特征被丢弃，'passthrough'表示该列不做任何处理
  + columns：指定该转换器对应的列，str, array-like of str 时指定列名称，int, array-like of int时指定列的索引
+ `remainder`：指定剩余列如何处理
  + 'drop'，丢弃剩余列，这是**默认值**
  + 'passthrough'，不做任何处理
  + estimator，使用指定的估计器处理剩余列
  
此外，还有一个`make_column_transformer()`函数，用于快速创建。

In [14]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer

In [13]:
df = pd.DataFrame({
    'col1': ['a', 'b', 'a', 'b', 'a'],
    'col2': [1, 5, 2, 3, 4],
    'col3': [6, 7, 8, 9, 10]
})
df

,col1,col2,col3
0,a,1,6
1,b,5,7
2,a,2,8
3,b,3,9
4,a,4,10


In [24]:
transformers=[('one-hot', OneHotEncoder(), ['col1']), ('min-max', MinMaxScaler(), ['col2'])]
ctf = ColumnTransformer(transformers=transformers, remainder='drop')

In [25]:
ctf.fit(df)

ColumnTransformer(transformers=[('one-hot', OneHotEncoder(), ['col1']),
                                ('min-max', MinMaxScaler(), ['col2'])])

In [26]:
ctf.transformers

[('one-hot', OneHotEncoder(), ['col1']), ('min-max', MinMaxScaler(), ['col2'])]

In [27]:
# col1 列使用 one-hot编码，产生2个特征，col2列只是缩放，仍然输出一个特征，col3没有指定转换器，处理方式为drop，所以最后的输出中被丢弃了
# 所以最后输出 3 列
ctf.transform(df)

array([[1.  , 0.  , 0.  ],
       [0.  , 1.  , 1.  ],
       [1.  , 0.  , 0.25],
       [0.  , 1.  , 0.5 ],
       [1.  , 0.  , 0.75]])

In [23]:
# col1 列使用 one-hot编码，产生2个特征，col2列只是缩放，仍然输出一个特征，col3没有指定转换器，处理方式为passthrough，所以最后的输出保持不变
# 所以最后输出 4 列
ctf = ColumnTransformer(transformers=transformers, remainder='passthrough')
ctf.fit_transform(df)

array([[ 1.  ,  0.  ,  0.  ,  6.  ],
       [ 0.  ,  1.  ,  1.  ,  7.  ],
       [ 1.  ,  0.  ,  0.25,  8.  ],
       [ 0.  ,  1.  ,  0.5 ,  9.  ],
       [ 1.  ,  0.  ,  0.75, 10.  ]])

# 封装自定义转换

如果要实现自定义转换，则需要使用`sklearn.preprocess`包中的`FunctionTransformer`类，对自定义处理流程进行包装。

`FunctionTransformer(func=None, inverse_func=None, *, validate=False, accept_sparse=False, check_inverse=True, feature_names_out=None, kw_args=None, inv_kw_args=None)`
+ `func`，自定义的处理流程函数
+ `inverse_func`，逆变换函数，可以为None，此时逆变换为等值变换
+ `kw_args`，dict, default=None，传递给自定义`func`的参数

注意，**这样封装的变换一般是stateless的，比如单纯的对数转换之类的，不需要保存状态**。

# 实现自定义转换器

# 特征融合

对同一组特征 **并行** 使用 $N$ 个转换器，然后将得到的 $N$ 个输出拼接成新的特征。  

使用`sklearn.pipeline`模块中的`FeatureUnion`类：   
`FeatureUnion(transformer_list, *, n_jobs=None, transformer_weights=None, verbose=False)`
+ `transformer_list`：list of (str, transformer) tuples，指定转换器，之后这些转换器的输出会被拼接起来。

还有一个`make_union()`函数方便使用。

In [28]:
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA, TruncatedSVD

In [35]:
X = [[0., 1., 3], [2., 2., 5]]

# PCA 保留2个特征， SVD 保留2个特征，最终的输出是 2+2=4 个特征
union = FeatureUnion([("pca", PCA(n_components=2)), ("svd", TruncatedSVD(n_components=2))])
union.fit(X)

FeatureUnion(transformer_list=[('pca', PCA(n_components=2)),
                               ('svd', TruncatedSVD())])

In [36]:
union.transform(X)

array([[ 1.50000000e+00, -1.11022302e-16,  3.03954967e+00,
         8.72432133e-01],
       [-1.50000000e+00,  1.11022302e-16,  5.72586357e+00,
        -4.63126787e-01]])

# 目标特征转换